In [1]:
from collections import defaultdict

In [2]:
# 1st element is favorite co-op, 2nd element is list of all co-ops
SEED = [
    ['Facebook', ['Facebook', 'Google', 'Figma', 'Coursera', 'Quora', 'Microsoft']],
    ['Yelp', ['Yelp', 'Monstercat', 'Noom', 'Klick', 'Sequence', 'Test']],
    ['Yelp', ['Snapchat', 'Facebook', 'Google', 'Uber', 'Pivotal']],
    ['Yelp', ['Snapchat', 'Facebook', 'Google', 'Uber', 'Pivotal']],
    ['Yelp', ['Snapchat', 'Facebook', 'Google', 'Uber', 'Pivotal']],
];

In [3]:
def rank_companies(companies):
    rankings = defaultdict(int)
    # Cleanse data
    for i, entry in enumerate(companies):
        # REMOVE DUPLICATES
        # People go back for different reasons: can't find a coop, or really
        # liked it. Since we don't have enough signal, we won't interpret the
        # results differently
        companies[i][1] = list(set(entry[1]))

    for i, entry in enumerate(companies):
        # Winner
        rankings[entry[0]] += 25
        for _, loser in enumerate(entry[1]):
            if loser != entry[0]:
                # LOSER
                rankings[loser] -= 25 / (len(entry[1]) - 1)
    return rankings

In [4]:
rank_companies(SEED)

defaultdict(int,
            {'Coursera': -5,
             'Facebook': 7,
             'Figma': -5,
             'Google': -23,
             'Klick': -5,
             'Microsoft': -5,
             'Monstercat': -5,
             'Noom': -5,
             'Pivotal': -18,
             'Quora': -5,
             'Sequence': -5,
             'Snapchat': -18,
             'Test': -5,
             'Uber': -18,
             'Yelp': 100})